In [ ]:
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup 

no_of_pages = 20
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

title = []
company = []
location = []
days_since_posted = []
link = []
career_level = []
qualification = []
years_of_experience = []
job_type = []
specialization = []
sql = []
excel = []
tableau = []
python = []
extracted_info = []
cols = ['title', 'company', 'days_since_posted', 'link', 'career_level', 'qualification',
        'years_of_experience', 'job_type', 'specialization', 'sql', 'excel', 'tableau', 'python']
    
def jobs_extract(job_list):    
    for job_post in job_list:
        # Job Title
        title.append(job_post.find('div', attrs={'class': 'z1s6m00 l3gun70 l3gun74 l3gun72'}).text.strip())

        # Company Name
        # sometimes data-automation attribute is not working
#         company.append(job_post.find('a', attrs={'data-automation': 'jobCardCompanyLink'}).text.strip()) 
        company.append(job_post.find('span', attrs={'class': 'z1s6m00 _17dyj7u1 _1hbhsw64u _1hbhsw60 _1hbhsw6r'}).text.strip())

        # Location
#         location = job_post.find('a', attrs={'data-automation': 'jobCardLocationLink'}).text
        location.append(job_post.find('span', attrs={'class': 'z1s6m00 _1hbhsw64u y44q7i0 y44q7i3 y44q7i21 y44q7ih'}).text)

        # Days since posted
        days_since_posted.append(job_post.find('time', attrs={'class': 'z1s6m00 _1hbhsw64u'}).text.strip())
        
        # Full details
        details = job_post.find('a', attrs={'class': 'jdlu994 jdlu996 jdlu999 y44q7i2 z1s6m00 z1s6m0f _1hbhsw6h'}, href=True)

        job_post_url = 'https://www.jobstreet.com.ph' + str(details['href'])
        job_post_page = requests.get(job_post_url, headers=headers)
        link.append(job_post_url)
        
        job_post_soup1 = BeautifulSoup(job_post_page.content,"html.parser")
        job_post_soup2 = BeautifulSoup(job_post_soup1.prettify(), "html.parser")

        job_post_div = job_post_soup2.findAll('div', attrs={'data-automation': 'jobDescription'})
        
        for job_desc in job_post_div:
            skills = job_desc.text

            if "sql" in skills.lower():
                sql.append("Yes")
            else:
                sql.append("No")
                
            if "excel" in skills.lower():
                excel.append("Yes")
            else:
                excel.append("No")
                
            if "tableau" in skills.lower():
                tableau.append("Yes")
            else:
                tableau.append("No")
                
            if "python" in skills.lower():
                python.append("Yes")   
            else:
                python.append("No")
        
        # Additional job information
        job_post_div2 = job_post_soup2.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw6r pmwfa50 pmwfa57'})
        
        addtl_desc = ['none', 'none', 'none', 'none', 'none']
        for job_addtl_desc in job_post_div2:
            skills2 = job_addtl_desc.text

            if "career level" in skills2.lower():
                pattern = re.compile("Career Level", re.IGNORECASE)
                addtl_desc[0] = pattern.sub("", skills2).strip()             
            elif "qualification" in skills2.lower():
                pattern = re.compile("Qualification", re.IGNORECASE)
                addtl_desc[1] = pattern.sub("", skills2).strip()             
            elif "years of experience" in skills2.lower():
                pattern = re.compile("Years of Experience", re.IGNORECASE)
                addtl_desc[2] = pattern.sub("", skills2).strip()             
            elif "job type" in skills2.lower():
                pattern = re.compile("Job Type", re.IGNORECASE)
                addtl_desc[3] = pattern.sub("", skills2).strip()             
            elif "job specializations" in skills2.lower():
                pattern = re.compile("Job Specializations", re.IGNORECASE)
                addtl_desc[4] = pattern.sub("", skills2).strip()
                
        #Save all additional information
        career_level.append(addtl_desc[0])
        qualification.append(addtl_desc[1])
        years_of_experience.append(addtl_desc[2])
        job_type.append(addtl_desc[3])
        specialization.append(addtl_desc[4])
    
def jobs_save():
    #Save all information
    all_jobs_list = {}

    extracted_info.append(title)
    extracted_info.append(company)
    extracted_info.append(days_since_posted)
    extracted_info.append(link)
    extracted_info.append(career_level)
    extracted_info.append(qualification)
    extracted_info.append(years_of_experience)
    extracted_info.append(job_type)
    extracted_info.append(specialization)
    extracted_info.append(sql)
    extracted_info.append(excel)
    extracted_info.append(tableau)
    extracted_info.append(python)    

    for j in range(len(cols)):
        all_jobs_list[cols[j]] = extracted_info[j]

    
    filename = "jobstreet_dataset.xls"
    jobs = pd.DataFrame(all_jobs_list)
    jobs.to_excel(filename, index=False)    
                                            
def jobs_search():
    for page in range(no_of_pages):
        # url = 'https://www.indeed.com.ph/jobs?q=' + skill + '&l=' + place + '&sort=date&start=' + str(page * 10)
        url = 'https://www.jobstreet.com.ph/en/job-search/data-analyst-jobs-in-philippines/' + str(page)
        page = requests.get(url, headers=headers)
        soup1 = BeautifulSoup(page.content,"html.parser")
        soup2 = BeautifulSoup(soup1.prettify(), "html.parser")

        job_list = soup2.findAll('div', attrs={'class': 'z1s6m00 _1hbhsw69e _1hbhsw68e _1hbhsw672 _1hbhsw67e'})
#         job_list = soup2.findAll('div', attrs={'data-automation': 'jobListing'})
        
        jobs_extract(job_list)
    
    jobs_save()
                
jobs_search()